In [1]:
import pandas as pd
import numpy as np


# Import Y

In [3]:
Y=pd.read_csv('comm-at-least-4-Y.csv',header=0,index_col=0)
Y=Y.sort_index(axis=0, level=None, ascending=True)
Y=Y.fillna(value=0)

In [227]:
#Transform to binary int
Y_label=np.asarray(Y['Local'], dtype="float64")

In [216]:
print len(Y_label)
print Y_label.sum()

2015
690.0



# Import and Transform X

In [5]:
likeall=pd.read_csv("./comm-at-least-4-engage.txt",header=None, names=['time','pageid','uid'],sep=' ')
likeall['count']=1

In [6]:
new_metric=likeall.pivot_table(index='uid',columns='pageid',values='count',aggfunc=np.sum)

In [7]:
#Save metric
new_metric.to_csv(path_or_buf='./comm-at-least-4-engage-metric.csv')

In [9]:
new_metric=new_metric.fillna(value=0)

# Feature Selection: Variance

In [5]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.09)
metric_variance=selector.fit_transform(new_metric)

ValueError: Found array with 0 sample(s) (shape=(0, 0)) while a minimum of 1 is required.

In [21]:
print new_metric.shape
print metric_variance.shape

(2015, 895)
(2015L, 67L)


In [38]:
new_columns=new_metric.columns[selector.variances_>=0.09]

In [39]:
np.savetxt('bayes-pages.txt',new_columns,fmt='%s')

In [226]:
X_selected=metric_variance

In [12]:
X_selected.shape

(2015L, 67L)

# Split Test Set

In [259]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(metric_variance, Y, test_size=200, random_state=42)

In [261]:
print X_test.shape, y_test.shape,X_train.shape,y_train.shape

(200L, 67L) (200, 3) (1815L, 67L) (1815, 3)


In [ ]:
X_selected=X_train
Y_label=np.asarray(Y['Yellow'], dtype="float64")

## Over-sampling SMOTE

In [228]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(kind='borderline1',ratio=1.0)
X_resampled, y_resampled = sm.fit_sample(X_selected, Y_label)

In [229]:
print X_resampled.shape
print len(y_resampled[y_resampled==1])

(3872L, 67L)
1936


In [230]:
X_selected=X_resampled
Y_label=y_resampled

# Bayes

In [231]:
from sklearn.cross_validation import KFold
from sklearn.naive_bayes import MultinomialNB
NB=MultinomialNB()
k=20
kf = KFold(3872, n_folds=k,shuffle=True)
recall=0.0
rate=0.0
precision=0.0
f1=0
for train_index, test_index in kf:
    NB.partial_fit(X_selected[train_index], Y_label[train_index],classes=[0,1])
    X=NB.predict(X_selected[test_index])
    X=pd.concat([pd.DataFrame(X),pd.DataFrame(Y_label[test_index])],axis=1, ignore_index=True)
    X['count']=1
    Acc=X.pivot_table(index=0,columns=1,values='count',aggfunc=np.sum)
    Acc=Acc.fillna(value=0)
    if Acc.shape==(2,2):
        rate=rate+(float(Acc[0][0]+Acc[1][1])/Acc.sum().sum())
        recall=recall+(float(Acc[1][1])/float(Acc[1][0]+Acc[1][1]))
        precision=precision+(float(Acc[1][1])/float(Acc[0][1]+Acc[1][1]))
        f1=f1+2 * ((float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])) * (float(Acc[1][1])/float(Acc[1][0]+Acc[1][1]))) / ((float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])) + (float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])))
    elif Acc.shape==(1,2):
        if Acc.index[0]==0:
            rate=rate+(float(Acc[0][0])/Acc.sum().sum())
        else:
            rate=rate+(float(Acc[0][1])/Acc.sum().sum())
            recall=recall+1
            precision=precision+(float(Acc[0][1])/Acc.sum().sum())
            f1=f1+(2 * ((float(Acc[0][1])/Acc.sum().sum()) * 1) / ((float(Acc[0][1])/Acc.sum().sum()) + 1))
    elif Acc.shape==(2,1):
        if Acc.columns==1:
            rate=rate+(float(Acc.iloc[1])/Acc.sum().sum())
            recall=recall+(float(Acc.iloc[1])/Acc.sum().sum())
            precision=precision+1
            f1=f1+1
        else:
            rate=rate+(float(Acc.iloc[0])/Acc.sum().sum())
            recall=recall+1
            precision=precision+1
            f1=f1+1
    else: #Acc.shape=(1,1)
        rate=rate+1
        recall=recall+1
        precision=precision+1
        f1=f1+1
rate=float(rate)/k
recall=float(recall)/k
precision=float(precision)/k
f1=float(f1)/k
print rate,f1,recall,precision

0.729117034346 0.759930046515 0.858757027307 0.683565742776


In [232]:
print NB.feature_log_prob_[1] #P(X|y) when y=1
rank = NB.feature_log_prob_[1].argsort()
print new_columns[rank].shape

[-11.99189391  -3.73658306  -4.15785613  -5.69450281 -11.99189391
  -3.76685428  -1.84429489  -4.57600746 -11.99189391 -11.99189391
  -4.21723746  -3.37807256  -3.47735261  -5.91799047 -11.99189391
 -11.99189391 -11.99189391  -5.56304752  -4.30666987 -11.99189391
  -3.80719747 -11.99189391  -4.71503166  -4.43952583  -5.57097444
  -5.3875759   -3.75599167  -5.60446915 -11.99189391  -5.04261443
  -4.67644803  -3.99254906  -3.01105096  -2.88547435  -3.0539085
  -6.25197378  -3.89633986  -3.74828535  -7.10123569  -3.27003511
  -5.9256664   -3.40585495  -4.01841304  -2.58374098 -11.99189391
  -3.81643309  -5.08216237  -5.69450281 -11.99189391  -3.42902451
  -5.49468991  -4.92736662 -11.99189391 -11.99189391 -11.99189391
 -11.99189391 -11.99189391  -4.8819484  -11.99189391  -3.70582603
  -3.12278927  -6.29333313  -6.58121515  -6.53985586  -3.48414425
 -11.99189391 -11.99189391]
(67L,)


In [ ]:
np.savetxt('bayes-pages-l.txt',new_columns[rank],fmt='%s')

# Predicting

In [192]:
print NB.feature_count_[0].sum()

219070.0


In [233]:
X_log_prob=np.log(X_selected.sum(axis=0)/X_selected.sum())

In [234]:
rank2=(NB.feature_log_prob_[1]-X_log_prob).argsort()

In [235]:
new_columns[rank2]

Int64Index([ 107065679322731,     126261789615,  544241848968882,
             108591025857376,      46446727755,  136845026417486,
                171502099497,  108930682554176,  121625551334730,
             522230507809373,  834591649886233,  497786323638932,
             423210364471433,  560688430610088,  172626052751159,
            1476654739234157,  517394455028454,  535289793202478,
                  5852261029,  544863235566535,  498565150232596,
             486392624735448,  156468627834558,  276842442361916,
             706305859460576,  512700315470576,  223783954322429,
                 46251501064,  169360916432640,     291139551002,
             587874824557675,  352962731493606,  634339023261344,
             149986161704699,  178965725523592,  117969648299869,
             494307913997852,     180447291099,  273813022746352,
             287013041410957,  139543009419663,     100375550229,
                 27743445015,  448027308647829,  160696287290644,
          

In [ ]:
np.savetxt('bayes-pages-b.txt',new_columns[rank2],fmt='%s')

In [3]:
commall=pd.read_csv("commall.csv",header=None, names=['time','pageid','uid'])

In [61]:
comm_filter=commall[commall.pageid.isin(new_columns)]

In [63]:
comm_filter['count']=1

C:\Users\junior\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [64]:
predict_metric=comm_filter.pivot_table(index='uid',columns='pageid',values='count',aggfunc=np.sum)

In [256]:
len(predict_metric)

368779

In [76]:
predict_metric=predict_metric.fillna(value=0)

In [ ]:
predict_results_local,predict_metric_local,pages_local
predict_results_yellow,predict_metric_yellow,pages_yellow
predict_results_blue,predict_metric_blue,pages_blue

In [129]:
predict_results_blue=NB.predict(predict_metric)

In [130]:
predict_results_blue.sum()

44270

In [131]:
predict_metric_blue=predict_metric.mul(predict_results_blue,axis='index')

In [132]:
pages_blue=predict_metric_blue.sum()

In [135]:
pages_blue.sort_values()

pageid
179264152235008         0.0
136845026417486         0.0
352962731493606         0.0
423210364471433         0.0
291139551002            0.0
494307913997852         1.0
544863235566535         1.0
149986161704699         1.0
108591025857376         1.0
535289793202478         2.0
127069410651529         2.0
171502099497            2.0
46251501064             2.0
406156242829341         3.0
560688430610088         3.0
548856635131690         3.0
729586907148929         3.0
569337929781203         4.0
544241848968882         4.0
497786323638932         5.0
517394455028454         6.0
172626052751159         7.0
144189602310884         7.0
223783954322429         9.0
46446727755             9.0
272574169481780        11.0
105759983026           11.0
108930682554176        11.0
117969648299869        11.0
126261789615           12.0
                     ...   
121358481388309        49.0
276842442361916        68.0
139543009419663        75.0
568718043162016        76.0
4982030902398

In [139]:
pages=pd.concat([pages_yellow,pages_blue,pages_local], axis=1)

In [160]:
pages_prob=pages.mul(1/pages.sum(axis=1),axis='index')

In [163]:
pages_entropy=pages_prob.mul(-np.log2(pages_prob),axis='index')

In [166]:
pages_entropy=pages_entropy.fillna(value=0)

In [253]:
pages_prob.sort_values(2,axis=0)

,0,1,2
pageid,,,
171502099497,0.866667,0.133333,0.000000
522230507809373,0.009798,0.988622,0.001580
834591649886233,0.014113,0.984207,0.001680
517394455028454,0.994040,0.003974,0.001987
706305859460576,0.005105,0.990059,0.004836
126261789615,0.987066,0.007055,0.005879
498565150232596,0.011428,0.982063,0.006510
107065679322731,0.249759,0.743587,0.006653
512700315470576,0.012447,0.980300,0.007252


In [180]:
pages_prob.loc[126261789615]

0    0.987066
1    0.007055
2    0.005879
Name: 126261789615, dtype: float64

In [244]:
pages_entropy.sum(axis=1)

pageid
5852261029          1.335909
10795878252         1.009484
27743445015         1.081821
46251501064         1.034259
46446727755         0.668515
100375550229        1.075382
105259197447        1.080485
105759983026        0.397632
126261789615        0.112523
171502099497        0.566510
180447291099        1.046618
196583419888        1.226883
269056797871        1.216801
291139551002        0.773227
107065679322731     0.865808
108591025857376     0.291549
108930682554176     0.700520
117969648299869     1.014018
121358481388309     1.067004
121625551334730     0.409913
127069410651529     1.000495
136845026417486     0.625262
139543009419663     1.117874
144189602310884     1.013603
149986161704699     0.584243
156468627834558     0.352140
160696287290644     1.194574
169360916432640     1.156472
172626052751159     0.844273
178965725523592     0.843700
                      ...   
273813022746352     0.802093
276842442361916     0.513707
287013041410957     0.937722
3529627

In [249]:
predict_metric.sum()

pageid
5852261029          32560.0
10795878252          7877.0
27743445015          5522.0
46251501064         23805.0
46446727755          5303.0
100375550229        12843.0
105259197447        75595.0
105759983026         9069.0
126261789615        20463.0
171502099497         3976.0
180447291099        11834.0
196583419888         3138.0
269056797871         3457.0
291139551002         5103.0
107065679322731     12787.0
108591025857376      4900.0
108930682554176      4702.0
117969648299869      3061.0
121358481388309      2857.0
121625551334730      2811.0
127069410651529      1904.0
136845026417486     27717.0
139543009419663      4371.0
144189602310884      2135.0
149986161704699     13003.0
156468627834558     20593.0
160696287290644      7329.0
169360916432640      8947.0
172626052751159      2479.0
178965725523592      4603.0
                     ...   
273813022746352     20923.0
276842442361916      3359.0
287013041410957     22908.0
352962731493606     25350.0
3738897226727

In [250]:
plot_datas=pd.concat([pages_entropy.sum(axis=1),predict_metric.sum()], axis=1)

In [254]:
pages_prob.sort_values(2,axis=0).to_csv(path_or_buf='pages-prob.csv')